<a href="https://colab.research.google.com/github/ychoi-kr/chatgpt-langchain/blob/main/chapter4/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4. 랭체인 기초

In [1]:
import os

os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

In [2]:
# 코랩에 보안 비밀을 등록한 경우
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

## 4-1 랭체인 개요

In [3]:
!pip install langchain==0.1.14 openai==1.14.3 langchain-openai==0.1.1

## 4-2 Language models

### LLMs

In [4]:
from langchain_openai import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0)
result = llm.invoke("자기소개를 해주세요.")
print(result)



안녕하세요. 저는 김지원입니다. 저는 대학교에서 경영학을 전공하고 현재는 기업에서 인사담당으로 일하고 있습니다. 저는 새로운 도전을 좋아하고 적극적으로 일에 임하는 성격을 가지고 있습니다. 또한 문제 해결능력과 커뮤니케이션 능력이 뛰어나며 적응력이 높은 편입니다. 제가 맡은 일에 대해서는 책임감을 가지고 최선을 다하며, 항상 성장하고 발전하는 것을 목표로 삼고 있습니다. 또한 다양한 사람들과 함께 일하며 새로운 아이디어를 공유하고 발전시키는 것을 즐기는 편입니다. 이러한 제


### Chat models

In [5]:
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="안녕하세요! 저는 존이라고 합니다!"),
    AIMessage(content="안녕하세요, 존 씨! 어떻게 도와드릴까요?"),
    HumanMessage(content= "제 이름을 아세요?")
]

result = chat.invoke(messages)
print(result.content)

네, 앞서 말씀하신대로 존 씨 맞으시죠?


### Callback을 이용한 스트리밍

In [6]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

messages = [HumanMessage(content="자기소개를 해주세요")]
result = chat.invoke(messages)

안녕하세요! 저는 인공지능 언어모델을 담당하는 AI입니다. 자연어 처리 기술을 활용하여 사용자들과 대화를 나누고 다양한 정보를 제공하는 것을 목표로 하고 있습니다. 질문이나 이야기를 주시면 최대한 도와드리겠습니다. 함께 즐거운 시간 보내요!

## 4-3 Prompts

### Prompt templates

In [7]:
from langchain.prompts import PromptTemplate

template = """
다음 요리의 레시피를 생각해 주세요.

요리: {dish}
"""

prompt = PromptTemplate(
   input_variables=["dish"],
   template=template,
)

result = prompt.format(dish="카레")
print(result)


다음 요리의 레시피를 생각해 주세요.

요리: 카레



### ChatPromptTemplate

In [8]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage

chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("당신은 {country} 요리 전문가입니다."),
    HumanMessagePromptTemplate.from_template("다음 요리의 레시피를 생각해 주세요.\n\n요리: {dish}")
])

messages = chat_prompt.format_prompt(country="영국", dish="고기감자조림").to_messages()

print(messages)

[SystemMessage(content='당신은 영국 요리 전문가입니다.'), HumanMessage(content='다음 요리의 레시피를 생각해 주세요.\n\n요리: 고기감자조림')]


In [9]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
result = chat.invoke(messages)
print(result.content)

고기감자조림 레시피:

재료:
- 쇠고기 (양지머리 또는 등심) 300g
- 감자 2개
- 양파 1개
- 당근 1/2개
- 대파 1대
- 마늘 3쪽
- 고추 1개 (선택사항)
- 간장 3큰술
- 설탕 1큰술
- 다진 마늘 1큰술
- 참기름 1큰술
- 식용유 2큰술
- 물 1컵

조리법:
1. 쇠고기는 한입 크기로 썰어 소금과 후추로 조미한 후 냉장고에서 30분 정도 숙성시킨다.
2. 감자, 양파, 당근은 깍뚝썰기로 썰고 대파는 어슷썰기로 준비한다.
3. 팬에 식용유를 두르고 다진 마늘을 볶다가 쇠고기를 넣고 익힌다.
4. 쇠고기가 익으면 감자, 양파, 당근을 넣고 볶는다.
5. 간장, 설탕, 물을 넣고 뚜껑을 덮어 중불에서 10분 정도 끓인다.
6. 대파와 고추를 넣고 한번 더 볶아 마무리한다.
7. 참기름을 뿌려 완성한다.

맛있는 고기감자조림이 완성되었습니다. 식사하실 때 즐겁게 드세요!


## 4-4 Output Parsers


### PydanticOutputParser

In [10]:
from pydantic import BaseModel, Field

class Recipe(BaseModel):
   ingredients: list[str] = Field(description="ingredients of the dish")
   steps: list[str] = Field(description="steps to make the dish")

In [11]:
from langchain.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=Recipe)

In [12]:
format_instructions = parser.get_format_instructions()

print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```


In [13]:
from langchain.prompts import PromptTemplate

template = """다음 요리의 레시피를 생각해 주세요.

{format_instructions}

요리: {dish}
"""

prompt = PromptTemplate(
   template=template,
   input_variables=["dish"],
   partial_variables={"format_instructions": format_instructions}
)

In [14]:
formatted_prompt = prompt.format(dish="카레")

print(formatted_prompt)

다음 요리의 레시피를 생각해 주세요.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```

요리: 카레



In [15]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
messages = [HumanMessage(content=formatted_prompt)]
output = chat.invoke(messages)

print(output.content)

{
  "ingredients": [
    "카레 가루",
    "양파",
    "감자",
    "당근",
    "고기 (소고기, 닭고기, 돼지고기 중 선택)",
    "물",
    "식용유",
    "소금",
    "후추"
  ],
  "steps": [
    "양파, 감자, 당근을 깍뚝 썰어준다.",
    "고기를 잘게 썰어준다.",
    "냄비에 식용유를 두르고 양파를 볶다가 고기를 넣고 익힌다.",
    "감자와 당근을 넣고 볶아준다.",
    "물을 부어 카레 가루를 넣고 끓인다.",
    "소금과 후추로 간을 해준다.",
    "쌀밥과 함께 내놓아 맛있게 즐긴다."
  ]
}


In [16]:
recipe = parser.parse(output.content)
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['카레 가루', '양파', '감자', '당근', '고기 (소고기, 닭고기, 돼지고기 중 선택)', '물', '식용유', '소금', '후추'] steps=['양파, 감자, 당근을 깍뚝 썰어준다.', '고기를 잘게 썰어준다.', '냄비에 식용유를 두르고 양파를 볶다가 고기를 넣고 익힌다.', '감자와 당근을 넣고 볶아준다.', '물을 부어 카레 가루를 넣고 끓인다.', '소금과 후추로 간을 해준다.', '쌀밥과 함께 내놓아 맛있게 즐긴다.']


## 4-5 Chains

### LLMChain―PromptTemplate, Language model, OutputParser를 연결

In [17]:
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field

class Recipe(BaseModel):
   ingredients: list[str] = Field(description="ingredients of the dish")
   steps: list[str] = Field(description="steps to make the dish")

output_parser = PydanticOutputParser(pydantic_object=Recipe)

template = """다음 요리의 레시피를 생각해 주세요.

{format_instructions}

요리: {dish}
"""

prompt = PromptTemplate(
   template=template,
   input_variables=["dish"],
   partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [18]:
from langchain.chains import LLMChain

chain = LLMChain(prompt=prompt, llm=chat, output_parser=output_parser)

recipe = chain.invoke("카레")

print(type(recipe))
print(recipe)

<class 'dict'>
{'dish': '카레', 'text': Recipe(ingredients=['카레 가루', '양파', '감자', '당근', '고기 (소고기, 닭고기, 돼지고기 중 선택)', '물', '식용유', '소금', '후추'], steps=['1. 양파, 감자, 당근을 깍뚝 썰어준다.', '2. 냄비에 식용유를 두르고 양파를 볶아준다.', '3. 고기를 넣고 익힌다.', '4. 감자와 당근을 넣고 볶아준다.', '5. 물을 부어 카레 가루를 넣고 끓인다.', '6. 소금과 후추로 간을 맞춰준다.', '7. 밥 위에 카레를 올려 맛있게 즐긴다.'])}


### SimpleSequentialChain ― Chain과 Chain을 연결

In [19]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

cot_template = """다음 질문에 답하세요.

질문: {question}

단계별로 생각해 봅시다.
"""

cot_prompt = PromptTemplate(
   input_variables=["question"],
   template=cot_template,
)

cot_chain = LLMChain(llm=chat, prompt=cot_prompt)

In [20]:
summarize_template = """다음 문장을 결론만 간단히 요약하세요.

{input}
"""
summarize_prompt = PromptTemplate(
   input_variables=["input"],
   template=summarize_template,
)

summarize_chain = LLMChain(llm=chat, prompt=summarize_prompt)

In [21]:
from langchain.chains import SimpleSequentialChain

cot_summarize_chain = SimpleSequentialChain(chains=[cot_chain, summarize_chain])

result = cot_summarize_chain.invoke(
   "저는 시장에 가서 사과 10개를 샀습니다. 이웃에게 2개, 수리공에게 2개를 주었습니다. 그런 다음에 사과 5개를 더 사서 1개를 먹었습니다. 남은 개수는 몇 개인가요?"
)
print(result["output"])

총 10개의 사과가 남았다.


위 입력은 [Chain-of-Thought Prompting | Prompt Engineering Guide](https://www.promptingguide.ai/techniques/cot)에서 인용했습니다.

## 4-6 Memory

### ConversationBufferMemory

In [22]:
from langchain.chains import ConversationChain
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory

chat = ChatOpenAI(model_name="gpt-4", temperature=0)
conversation = ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory()
)

while True:
    user_message = input("You: ")

    if user_message == "끝":
        print("(대화 종료)")
        break

    ai_message = conversation.invoke(input=user_message)["response"]
    print(f"AI: {ai_message}")

You: 안녕하세요. 저는 존이라고 합니다!
AI: 안녕하세요, 존님! 저는 인공지능 대화형 도우미입니다. 어떻게 도와드릴까요?
You: 제 이름을 아세요?
AI: 네, 방금 말씀하셨듯이 당신의 이름은 존님이라고 알고 있습니다.
You: 끝
(대화 종료)


### (칼럼) Chat models에서 Memory를 사용할 때 주의할 점

In [23]:
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="안녕하세요. 저는 존이라고 합니다!"),
    AIMessage(content="안녕하세요, 존 님! 어떻게 도와드릴까요?"),
    HumanMessage(content= "제 이름을 아세요?")
]

result = chat.invoke(messages)
print(result.content)

네, 앞서 말씀해주신대로 존 님의 이름을 알고 있습니다. 어떻게 도와드릴까요?
